In [1]:
%cd ..
import os
import time
import json

/home/yongming/apperception


In [2]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects
from optimized_ingestion.utils.preprocess import preprocess
database.connection
from optimized_ingestion.cache import disable_cache
disable_cache()

YOLOv5 🚀 2023-4-10 Python-3.10.6 torch-2.0.0+cu117 CUDA:0 (Tesla T4, 14966MiB)

Using cache found in /home/yongming/apperception/weights/ultralytics_yolov5_master


Using cuda:0


requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Defaulting to user installation because normal site-packages is not writeable

requirements: 1 package updated per /home/yongming/apperception/weights/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-4-10 Python-3.10.6 torch-2.0.0+cu117 CUDA:0 (Tesla T4, 14966MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"
if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/data/processed/full-dataset/trainval"
NUSCENES_RAW_DATA = "NUSCENES_RAW_DATA"
if NUSCENES_RAW_DATA in os.environ:
    RAW_DATA_DIR = os.environ[NUSCENES_RAW_DATA]
else:
    RAW_DATA_DIR = "/data/full-dataset/trainval"

In [4]:
video_set = []

In [5]:
def bechmark_detection_estimation_gain(world,video_names=[], path_suffix=None):
    base_benchmark_path = f'./outputs/base_pipeline_benchmark{"_"+path_suffix if path_suffix else ""}.json'
    ### base pipeline
    preprocess(world, DATA_DIR, video_names,
               benchmark_path=base_benchmark_path)
    ### detection estimation benchmark
    for skip_ratio in [0.1 * i for i in range(1,10)]:
        print(f"current skip ratio{skip_ratio}")
        optimize_benchmark_path = f'./outputs/detection_estimation_pipeline_{int(skip_ratio*10)}{"_"+path_suffix if path_suffix else ""}.json'
        preprocess(world, DATA_DIR, video_names,
                   base=False,
                   benchmark_path=optimize_benchmark_path,
                   skip_ratio=skip_ratio)
    

In [6]:
name = 'ScenicWorld' # world name
world = empty_world(name=name)

In [7]:
bechmark_detection_estimation_gain(world)

YOLOv5 🚀 2023-4-10 Python-3.10.6 torch-2.0.0+cu117 CUDA:0 (Tesla T4, 14966MiB)

requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Defaulting to user installation because normal site-packages is not writeable

requirements: 1 package updated per /home/yongming/apperception/weights/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-4-10 Python-3.10.6 torch-2.0.0+cu117 CUDA:0 (Tesla T4, 14966MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


scene-0103-CAM_FRONT --------------------------------------------------------------------------------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.33s/it]
YOLOv5 🚀 2023-4-10 Python-3.10.6 torch-2.0.0+cu117 CUDA:0 (Tesla T4, 14966MiB)

Downloading...
From: https://drive.google.com/uc?id=1Kkx2zW89jq_NETu4u42CFZTMVD5Hwm6e
To: /home/yongming/apperception/weights/osnet_x0_25_msmt17.pt
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9.34M/9.34M [00:00<00:00, 25.2MB/s]


Successfully loaded pretrained weights from "/home/yongming/apperception/weights/osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
NOTICE:  table "cameras" does not exist, skipping

NOTICE:  table "item_general_trajectory" does not exist, skipping

error video: scene-0103-CAM_FRONT with skip ratio 0
scene-0103-CAM_FRONT_RIGHT --------------------------------------------------------------------------------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.41s/it]
YOLOv5 🚀 2023-4-10 Python-3.10.6 torch-2.0.0+cu117 CUDA:0 (Tesla T4, 14966MiB)



...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.84s/it]
YOLOv5 🚀 2023-4-10 Python-3.10.6 torch-2.0.0+cu117 CUDA:0 (Tesla T4, 14966MiB)



Successfully loaded pretrained weights from "/home/yongming/apperception/weights/osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
NOTICE:  table "cameras" does not exist, skipping

NOTICE:  table "item_general_trajectory" does not exist, skipping

error video: scene-0757-CAM_BACK_LEFT with skip ratio 0
scene-0757-CAM_FRONT_LEFT --------------------------------------------------------------------------------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.64s/it]
YOLOv5 🚀 2023-4-10 Python-3.10.6 torch-2.0.0+cu117 CUDA:0 (Tesla T4, 14966MiB)



Successfully loaded pretrained weights from "/home/yongming/apperception/weights/osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
NOTICE:  table "cameras" does not exist, skipping

NOTICE:  table "item_general_trajectory" does not exist, skipping

error video: scene-0757-CAM_FRONT_LEFT with skip ratio 0
num_video:  24
total preprocess time 746.93252825737


FileNotFoundError: [Errno 2] No such file or directory: './outputs/base_pipeline_benchmark.json'

In [ ]:
obj1 = objects[0]
cam = camera
car_world = empty_world(name=name).filter(
    (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck') | F.like(obj1.type, 'bus'))
)

In [ ]:
bechmark_detection_estimation_gain(car_world, path_suffix="only_car")

In [ ]:
with open('./outputs/base_pipeline_benchmark.json') as benchmark_file:
    benchmark_content = benchmark_file.read()

parsed_json = json.loads(benchmark_content)
baseline_total_runtime = parsed_json[0]['total_runtime']
baseline_avg_runtime = parsed_json[1]['average runtime']
baseline_stage_runtimes = parsed_json[0]['stage_runtimes']
for stage_runtime in baseline_stage_runtimes:
    if stage_runtime['stage'] == 'Tracking2D.StrongSORT':
        baseline_strong_sort_runtime = sum([r['runtime'] for r in stage_runtime['runtimes']])

# num_videos = parsed_json[2]['number of videos']
print(baseline_total_runtime)
print(baseline_avg_runtime)
print(baseline_strong_sort_runetime)

In [1]:
skip_ratios = [i for i in range(1, 10)]
all_obj_total_runtimes = []
all_obj_avg_runtimes = []
all_obj_strong_sort_runtimes = []
only_car_total_runtimes = []
only_car_avg_runtimes = []
only_car_strong_sort_runtimes = []
for i in skip_ratios:
    with open(f'./outputs/detection_estimation_pipeline_{i}.json') as benchmark_file:
        benchmark_content = benchmark_file.read()

    parsed_json = json.loads(benchmark_content)
    all_obj_total_runtimes.append(parsed_json[0]['total_runtime'])
    all_obj_avg_runtimes.append(parsed_json[1]['average runtime'])
    all_obj_stage_runtimes = parsed_json[0]['staged_runtimes']
    for stage_runtime in all_obj_stage_runtimes:
        if stage_runtime['stage'] == 'Tracking2D.StrongSORT':
            all_obj_strong_sort_runtimes.append(sum([r['runtime'] for r in stage_runtime['runtimes']]))

    with open(f'./outputs/detection_estimation_pipeline_{i}_only_car.json') as benchmark_file:
        benchmark_content = benchmark_file.read()

    parsed_json = json.loads(benchmark_content)
    only_car_total_runtimes.append(parsed_json[0]['total_runtime'])
    only_car_avg_runtimes.append(parsed_json[1]['average runtime'])
    only_car_stage_runetimes = parsed_json[0]['staged_runtimes']
    for stage_runtime in only_car_stage_runetimes:
        if stage_runtime['stage'] == 'Tracking2D.StrongSORT':
            only_car_strong_sort_runtimes.append(sum([r['runtime'] for r in stage_runtime['runtimes']]))

print(all_obj_total_runtimes)
print(all_obj_avg_runtimes)
print(only_car_total_runtimes)
print(only_car_avg_runtimes)
print(all_obj_strong_sort_runtimes)
print(only_car_strong_sort_runtimes)

FileNotFoundError: [Errno 2] No such file or directory: './outputs/detection_estimation_pipeline_1.json'

In [ ]:
import matplotlib.pyplot as plt
print(f'number of videos {num_videos}')
fig, ax = plt.subplots()
ax.plot(skip_ratios, len(skip_ratios)*[baseline_total_runtime], label='baseline')
ax.plot(skip_ratios, all_obj_total_runtimes, label='all obj')
ax.plot(skip_ratios, only_car_total_runtimes, label='only car')
legend = ax.legend(loc='upper center', shadow=True)
plt.ylabel('total run time')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
print(f'number of videos {num_videos}')
fig, ax = plt.subplots()
ax.plot(skip_ratios, len(skip_ratios)*[baseline_total_runtime/baseline_total_runtime], label='baseline')
ax.plot(skip_ratios, [e/baseline_total_runtime for e in all_obj_total_runtimes], label='all obj')
ax.plot(skip_ratios, [e/baseline_total_runtime for e in only_car_total_runtimes], label='only car')
legend = ax.legend(loc='upper center', shadow=True)
plt.ylabel('total run time/baseline total run time')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
print(f'number of videos {num_videos}')
fig, ax = plt.subplots()
ax.plot(skip_ratios, len(skip_ratios)*[baseline_avg_runtime], label='baseline')
ax.plot(skip_ratios, all_obj_avg_runtimes, label='all obj')
ax.plot(skip_ratios, only_car_avg_runtimes, label='only car')
legend = ax.legend(loc='upper center', shadow=True)
plt.ylabel('avg run time')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
print(f'number of videos {num_videos}')
fig, ax = plt.subplots()
ax.plot(skip_ratios, len(skip_ratios)*[baseline_strong_sort_runetime], label='baseline')
ax.plot(skip_ratios, all_obj_strong_sort_runetime, label='all obj')
ax.plot(skip_ratios, only_car_strong_sort_runetime, label='only car')
legend = ax.legend(loc='upper center', shadow=True)
plt.ylabel('strongsort run time')
plt.show()